In [29]:
import gzip
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec, LdaMulticore

from gensim.corpora import Dictionary

import numpy as np
import pyLDAvis
from pyLDAvis import gensim

In [30]:
ls data

 Volume in drive C has no label.
 Volume Serial Number is 16EE-C352

 Directory of C:\Users\user\data

12/09/2019  23:04    <DIR>          .
12/09/2019  23:04    <DIR>          ..
12/09/2019  23:04       240,342,787 reviews_data.txt
               1 File(s)    240,342,787 bytes
               2 Dir(s)  39,307,505,664 bytes free


In [31]:
with open(r'data\reviews_data.txt', 'rb') as f:
    for i, line in enumerate(f):
        print(i,line)
        break
        

0 b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in 

In [32]:
#function -delete shallow, contains unnecessary words (STOPWORDS -list of that)
#and returns others words

def preprocesing(sentence):
    return [word for word in simple_preprocess(sentence) if word not in STOPWORDS]

#function -can read all the sentences by the line
def read_sentences(filename):
    with open(filename, 'rb') as f:
        for line in f:
            yield preprocesing(line)
        


In [33]:
#how long it's gonna take 
%time sentences = list(read_sentences( r'data\reviews_data.txt'))

Wall time: 1min 35s


In [34]:
len(sentences)

255404

# Model

In [35]:
%time model =  Word2Vec(sentences, size=100, window=5, min_count=2)

Wall time: 2min 13s


# Reprezentacja wektorowa

In [36]:
model.wv.most_similar('good')

[('decent', 0.8321042060852051),
 ('great', 0.8192640542984009),
 ('excellent', 0.8031087517738342),
 ('fair', 0.6543538570404053),
 ('ok', 0.6499290466308594),
 ('reasonable', 0.6371042728424072),
 ('average', 0.6278350353240967),
 ('terrific', 0.6231874227523804),
 ('nice', 0.6055673360824585),
 ('fantastic', 0.6049915552139282)]

In [37]:
model.wv.most_similar(positive=['bad', 'terrible'], negative=['good'])

[('horrible', 0.8376836776733398),
 ('awful', 0.8019347190856934),
 ('horrendous', 0.700883150100708),
 ('dreadful', 0.6944225430488586),
 ('shocking', 0.6922716498374939),
 ('horrific', 0.665151834487915),
 ('horrid', 0.6645398139953613),
 ('disgusting', 0.6627089977264404),
 ('appalling', 0.659607470035553),
 ('nasty', 0.6557696461677551)]

In [38]:
model.wv.most_similar('trustworthy')

[('absent', 0.5492411851882935),
 ('fiendly', 0.5416439771652222),
 ('unresponsive', 0.5414178371429443),
 ('amiable', 0.5388038158416748),
 ('houseman', 0.530662477016449),
 ('manning', 0.5275468826293945),
 ('operative', 0.5245778560638428),
 ('cooperative', 0.5201233625411987),
 ('trafficking', 0.5130311846733093),
 ('retrain', 0.5049230456352234)]

# Topic modeling - LDA

In [39]:
sentences_light = np.random.permutation(sentences)

In [40]:
sentences_light = sentences_light[:1000]

In [41]:
%time dictionary = Dictionary(sentences_light)

Wall time: 219 ms


In [42]:
#show how many words we  have in a 1000 sentences.
len(dictionary)

8786

In [43]:
#Bag of Words
%time bow_corpus = [dictionary.doc2bow(sent) for sent in sentences_light]

Wall time: 125 ms


In [44]:
#this number means 'token with id 515 apeared 2times in one sentence'
dictionary.doc2bow(['car', 'car'])

[(135, 2)]

In [45]:
#this number means 'token car has assigned  number 515'
dictionary.token2id['car']

135

In [46]:
#num_topics-number of topics tht we want, passes-how many times it will be repeated, workers=how many CPU
%time lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=100, passes=20, workers=8)

Wall time: 1min 8s


In [47]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"room" + 0.015*"service" + 0.014*"good" + 0.014*"stay" + 0.011*"chambermaid" + 0.010*"hotels" + 0.010*"little" + 0.009*"time" + 0.009*"tour" + 0.008*"pm"
Topic: 1 
Words: 0.027*"hotel" + 0.014*"room" + 0.012*"quot" + 0.011*"service" + 0.010*"good" + 0.009*"excellent" + 0.008*"choice" + 0.008*"india" + 0.008*"night" + 0.007*"card"
Topic: 2 
Words: 0.025*"hotel" + 0.013*"small" + 0.011*"stayed" + 0.011*"staff" + 0.010*"room" + 0.010*"good" + 0.009*"area" + 0.008*"location" + 0.008*"little" + 0.007*"stay"
Topic: 3 
Words: 0.020*"great" + 0.018*"stay" + 0.018*"room" + 0.017*"time" + 0.016*"stayed" + 0.016*"hotel" + 0.014*"clean" + 0.011*"rooms" + 0.010*"dec" + 0.010*"breakfast"
Topic: 4 
Words: 0.024*"hotel" + 0.019*"room" + 0.012*"good" + 0.011*"night" + 0.011*"location" + 0.010*"staff" + 0.010*"amp" + 0.007*"comfortable" + 0.007*"pm" + 0.007*"drink"
Topic: 5 
Words: 0.040*"room" + 0.018*"hotel" + 0.008*"quot" + 0.008*"bed" + 0.008*"stay" + 0.007*"location" + 0.007*

# Wizualizacja tematów

In [50]:
lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)